### moving from jinja to python.py prompts 
i though i will use jinja for prompts here but forget that  jinja is not made for routing , my prompts 
here looks less like template and more like dynamic routing prompts 

 i was using lots and lots of if else condition 
say for 20 agent , checking 20 if else condition make system slow so i am moving 


In [1]:

"""    
    Returns the prompt text for a given role and agent type.
    Fills in any required variables provided as kwargs.
    
    Parameters:
        role (str): 'system' or 'user'
        agent_type (str): e.g., 'validator_agent', 'researcher_agent'
        **kwargs: required variables for the prompt (if any)
        
    Returns:
        str: Fully rendered prompt text
    

"""

from prompt import get_prompt
from llm_clients import call_llm

/home/ujjwal/projects/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
index_name= 'genai-mas-mcp-01'
NAMESPACE_KNOWLEDGE = "knowledgeStore"
NAMESPACE_CONTEXT= "ContextLibrary"

## create an MCP 

In [3]:
from typing import Union
from pydantic import BaseModel , field_validator , Field 
from typing import Dict, Any , Union , TypedDict
import uuid


class ValidatorContext(BaseModel):
    task: str
    source_summary: str
    draft_post: str



class MCPMessage(BaseModel):
    protocol_version: str = "1.0"
    sender: str= Field(min_length=3)
    content: Any
    metadata: Dict[str, Any] = Field(default_factory=dict)


def create_mcp_message(sender: str ,  content: Any, metadata=None) -> MCPMessage:
    """Create a message in MCP format."""

    
    if not isinstance(content, (str, dict, ValidatorContext)):
        raise TypeError(
            f"content must be str, dict, or ValidatorContext; got {type(content).__name__}"
        )

        

    metadata = metadata or {} 
    metadata.setdefault("task_id", str(uuid.uuid4()))
    metadata.setdefault("parents",[])

    return MCPMessage(
        sender=sender,
        content=content,
        metadata=metadata   
    )

mcp_answer = create_mcp_message("agent_1", "Hello, this is a test message.", {"priority": "high" , "parents": ["writer-agent"]})
print(mcp_answer.sender)
print(mcp_answer.content)
print(mcp_answer.metadata)

agent_1
Hello, this is a test message.
{'priority': 'high', 'parents': ['writer-agent'], 'task_id': '062196b8-01bd-453f-8400-537c12776928'}


In [4]:
from helper import display_mcp 
display_mcp(mcp_answer)



--- MCP Message (Sender: agent_1) ---
Content: Hello, this is a test message.
Metadata Keys: ['priority', 'parents', 'task_id']
------------------------------------


####  not using any fallback logics or anything or simplicity 
#### i found i  will only use those if topic specific or  in end to end projects

In [5]:
import json 

In [8]:
def agent_context_librarian(mcp_message:MCPMessage):
    """
        Retrieves the appropriate Semantic Blueprint from the Context Library.
    """
    
    if not INDEX :
        raise("this except Pine cone index ")


    if "intent_query" not in mcp_message.content:
        raise KeyError("mcp_message['content'] must contain 'intent_query'")
    
    print("\\n[Librarian] Activated. Analyzing intent...")
    requested_intent = mcp_message.content['intent_query']
    results = query_pinecone(index= INDEX, query_text=requested_intent, namespace= NAMESPACE_CONTEXT, top_k=1)
    if results:
        match = results[0]
        print(f"[Librarian] Found blueprint '{match['id']}' (Score:{match['score']:.2f})")
        blueprint_json = match['metadata']['blueprint_json']
        content = {"blueprint": blueprint_json}

    else:
        print("[Librarian] No specific blueprint found. Returning default.")
        content = {"blueprint": json.dumps({"instruction": "Generate the content neutrally."})}
    return create_mcp_message("Librarian", content)



## research agent 


In [9]:
## pine cone query 
from helper import query_pinecone 
## takes  query_text , namespaces and tok k (optional, because defult is 1)
## and return index query ... response['matches']

In [10]:
INDEX_NAME = 'genai-mas-mcp-01'
NAMESPACE_KNOWLEDGE = "knowledgeStore"
NAMESPACE_CONTEXT = "ContextLibrary"



In [11]:
from helper import get_or_create_index
from llm_clients import pine_cone_client

index = get_or_create_index(
    pc=pine_cone_client,
    index_name=INDEX_NAME,
    embedding_dim=384,
    namespaces_to_clear=None  # important: keep existing vectors
)



In [12]:
# 2. Define test query
query_text = "create a suspense narrator story"

# 3. Query CONTEXT namespace (semantic blueprints live here)
context_results = query_pinecone(
    index=index,
    query_text=query_text,
    namespace=NAMESPACE_KNOWLEDGE,
    top_k=1
)


In [14]:
context_results

[{'id': 'knowledge_chunk_1',
  'metadata': {'text': 'nuclear power, demonstrating that solar energy can '
                       'support missions even at great distances from the Sun.  '
                       'Mars exploration represents another major focus of '
                       'modern space science. Robotic rovers act as mobile '
                       'laboratories, analyzing soil, rocks, and atmospheric '
                       'conditions. NASA’s Perseverance rover is specifically '
                       'designed to investigate whether Mars ever supported '
                       'microbial life, while also testing technologies for '
                       'future human missions. It carried the Ingenuity '
                       'helicopter, which achieved the first powered flight on '
                       'another planet, proving that aerial exploration is '
                       'possible beyond Earth.  Together, these missions show '
                       'how spa

In [15]:
[text['metadata']['text'] for text in context_results]

['nuclear power, demonstrating that solar energy can support missions even at great distances from the Sun.  Mars exploration represents another major focus of modern space science. Robotic rovers act as mobile laboratories, analyzing soil, rocks, and atmospheric conditions. NASA’s Perseverance rover is specifically designed to investigate whether Mars ever supported microbial life, while also testing technologies for future human missions. It carried the Ingenuity helicopter, which achieved the first powered flight on another planet, proving that aerial exploration is possible beyond Earth.  Together, these missions show how space exploration has evolved from political competition into a long-term scientific and technological endeavor aimed at expanding human knowledge and enabling future interplanetary travel.']

## agent research

In [16]:
from llm_clients import cohere_chat_client
from prompt import get_prompt 

In [17]:
query_pinecone

<function helper.query_pinecone(index, query_text: str, namespace: str, top_k: int = 1)>

In [27]:
def agent_researcher(mcp_message:MCPMessage):
    
    topic= mcp_message.content['topic_query']
    results= query_pinecone(index= INDEX , query_text=topic, namespace =NAMESPACE_KNOWLEDGE, top_k=3)
    if not results: 
        print("[researcher] no Relevant information found")
        return create_mcp_message("Researcher", {"facts" : "No data found"})
    
    source_texts = [match['metadata']['text'] for match in results]
    system_prompt= get_prompt(role="system", agent_type="research_synthesis_ai")
    
    user_prompt=  get_prompt(role="user", agent_type="research_synthesis_ai",topic= topic , sources= source_texts )
    
    findings= call_llm(system_prompt, user_prompt)
    
    return create_mcp_message("Researcher" , {"facts": findings})
    

## Writer agent

In [19]:
def agent_writer(mcp_message):
    
    print("\\n[Writer] Activated. Applying blueprint to facts...")
    facts= mcp_message.content['facts']
    blueprint_json_string= mcp_message.content['blueprint']
    
    system_prompt= get_prompt(role='system' , agent_type="writer_blueprint" , blueprint_json_string = blueprint_json_string)
    user_prompt= get_prompt(role="user",agent_type= "research_facts", facts= facts  )
    
    final_output= call_llm(system_prompt, user_prompt)
    return create_mcp_message("Writer", {"output": final_output})




# final (building the orchestrator)

In [20]:
from prompt import get_prompt 
system_prompt= get_prompt(role='system' , agent_type="goal_analyst" )
system_prompt

"You are an expert goal analyst.\nAnalyze the user's high-level goal and extract two components:\n1. 'intent_query': A descriptive phrase summarizing the desired style, tone, or format, optimized for searching a context library (e.g., 'suspenseful narrative blueprint', 'objective technical explanation structure').\n2. 'topic_query': A concise phrase summarizing the factual subject matter required (e.g., 'Juno mission objectives and power', 'Apollo 11 landing details').\nRespond ONLY with a JSON object containing these two keys."

## Goal Analysis

In [21]:
from llm_clients import call_llm
analysis_system_prompt = """You are an expert goal analyst.
                            Analyze the
                            user's high-level goal and extract two components:
                            1. 'intent_query': A descriptive phrase summarizing the desired style, tone,
                            or format, optimized for searching a context library (e.g., "suspenseful narrative
                            blueprint", "objective technical explanation structure").
                            2. 'topic_query': A concise phrase summarizing the factual subject matter
                            required (e.g., "Juno mission objectives and power", "Apollo 11 landing details").
                            Respond ONLY with a JSON object containing these two keys."""

In [22]:
answer= call_llm(system_prompt = analysis_system_prompt ,user_content=  "hellow", structured=True)
answer 

ResearcherOutput(intent_query='informal greeting style', topic_query='basic conversational phrases')

In [23]:
answer.model_dump()

{'intent_query': 'informal greeting style',
 'topic_query': 'basic conversational phrases'}

In [24]:
from llm_clients import pine_cone_client 
index_name= 'genai-mas-mcp-01'
NAMESPACE_KNOWLEDGE = "knowledgeStore"
NAMESPACE_CONTEXT= "ContextLibrary"

INDEX= get_or_create_index(pc = pine_cone_client , index_name= index_name)



In [28]:
def orchestrator(hight_level_goal):
    """ Manages the workflow of the context Aware MAS """ 
    
    print(f"high level goal :  {hight_level_goal}")
    system_prompt= get_prompt(role='system' , agent_type="goal_analyst" )
    
    analysis_results= call_llm(system_prompt, hight_level_goal, structured=True)
    
    try: 

        intent_query= analysis_results.intent_query 
        topic_query= analysis_results.topic_query
    except   : 
        return  "error occours while trying to fetch intent , topic query"
    
    # Agent coordination 
    mcp_to_librarian= create_mcp_message(
        sender= "Orchestration",
        content= {"intent_query" : intent_query} 
    )
    
    mcp_from_librarian= agent_context_librarian(mcp_to_librarian)
    
    ## display mcp 
    display_mcp(mcp_from_librarian ,"Librarian -> Orchestrator")
    
    context_blueprint = mcp_from_librarian.content.get('blueprint')
    
    if not context_blueprint:
        raise ValueError("No blueprint found in Librarian's content")

    ## step 2 
    mcp_to_researcher = create_mcp_message( sender="Orchestrator", content={"topic_query": topic_query})
    mcp_from_researcher=agent_researcher(mcp_to_researcher)
    display_mcp(mcp_from_researcher)
    
    research_finding= mcp_from_researcher.content.get('facts')
    if not research_finding :
        raise ValueError("no research finding ")
    
    ## generate final output , step 3
    writer_task = {
                        "blueprint": context_blueprint,
                        "facts": research_finding
                        }
    
    mcp_to_writer = create_mcp_message(
                                        sender="Orchestrator",
                                        content=writer_task
                                        )
    
                                            
    mcp_from_writer = agent_writer(mcp_to_writer)
    display_mcp(mcp_from_writer, "Writer -> Orchestrator")
    
    
    final_result = mcp_from_writer.content.get('output')
    
    return final_result 
    
    
    
    
    
    

In [29]:
result = orchestrator("Explain space exploration in a suspenseful story")
print(result)

high level goal :  Explain space exploration in a suspenseful story
\n[Librarian] Activated. Analyzing intent...
[Librarian] Found blueprint 'blueprint_suspense_narrative' (Score:0.39)

--- Librarian -> Orchestrator (Sender: Librarian) ---
Content Keys: ['blueprint']
Metadata Keys: ['task_id', 'parents']
--------------------------------------------------

--- MCP Message (Sender: Researcher) ---
Content Keys: ['facts']
Metadata Keys: ['task_id', 'parents']
------------------------------------
\n[Writer] Activated. Applying blueprint to facts...

--- Writer -> Orchestrator (Sender: Writer) ---
Content Keys: ['output']
Metadata Keys: ['task_id', 'parents']
-----------------------------------------------
**Shadows in the Void**  

The Agent stands in the dim glow of the control room. Screens flicker with data from distant worlds. The hum of machines fills the air, a steady rhythm in the silence. Outside, the vastness of space presses against the station’s walls, a darkness broken only by 

## THE END 